In [ ]:
print('start contract labels')
import sys
sys.path.append("../helper_functions")
import google_bq_utils as bqu
import clickhouse_utils as ch
sys.path.pop()

import numpy as np
import pandas as pd
import time
import requests as r

In [ ]:
api_configs = {
    'old_gtp': {
        'url': 'https://api.growthepie.xyz/v1/contracts.json',
        'table_name': 'contract_labels_growthepie'
    },
    'oli_gtp': {
        'url': 'https://api.growthepie.xyz/test/oli_labels.json',
        'table_name': 'oli_contract_labels_growthepie'
    },
    'oss': {
        'url': 'https://api.growthepie.xyz/test/oss_projects.json',
        'table_name': 'oss_project_labels_growthepie'
    }
}

In [ ]:
def fetch_and_process_data(url):
    response = r.get(url)
    data = response.json()
    df = pd.DataFrame(data)
    if 'address' in df.columns:
        df['address_original'] = df['address']
        df['address'] = df['address'].str.lower()

    # If 'is_factory_contract' exists, fill nulls with 'false'
    if 'is_factory_contract' in df.columns:
        df['is_factory_contract'] = df['is_factory_contract'].fillna('false')

    # Convert all object columns to strings
    object_columns = df.select_dtypes(include=['object']).columns
    df[object_columns] = df[object_columns].astype(str)
    return df

# Iterate through the configurations
for config_name, config in api_configs.items():
    df = fetch_and_process_data(config['url'])
    # You can now use df and config['table_name'] as needed
    print(f"Processed data for {config_name}:")
    print(f"Table name: {config['table_name']}")
    ch.write_df_to_clickhouse(df, config['table_name'], if_exists='replace')
    print('clickhouse complete')
    bqu.write_df_to_bq_table(df, config['table_name'])
    print('bq complete')
    print(f"Table name: {config['table_name']}")
    print(f"Number of rows: {len(df)}")
    print("---")